In [1]:
import os

os.environ["HF_HOME"] = "D:/hf_cache"
os.environ["HF_DATASETS_CACHE"] = "D:/hf_cache/datasets"
os.environ["HF_MODEL_CACHE"] = "D:/hf_cache/models"




In [2]:
!python -m pip install --upgrade --user pip
!pip install transformers --user
!pip install torch --user
!pip install torchvision torchaudio --user
!pip install accelerate --user
!pip install datasets --user

In [3]:
!pip install evaluate

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "auto"
model_path = "ibm-granite/granite-3.0-2b-base"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)
model.eval()

C:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.25it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49152, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): GraniteRMSNorm((2048,)

In [5]:
from datasets import load_dataset
dataset = load_dataset("UCSC-VLAA/MedReason")
print(dataset)

# Extract the training data
train_dataset = dataset['train']

# Display some sample data
print(train_dataset[0])

DatasetDict({
    train: Dataset({
        features: ['dataset_name', 'id_in_dataset', 'question', 'answer', 'reasoning', 'options'],
        num_rows: 32682
    })
})
{'dataset_name': 'medmcqa', 'id_in_dataset': 7131, 'question': 'Urogenital Diaphragm is made up of the following, except:', 'answer': "Colle's fascia. Explanation: Colle's fascia does not contribute to the Urogenital Diaphragm. It is attached posteriorly to the posterior border of the urogenital diaphragm but does not form pa of this diaphragm. Ref: BDC, Volume 2, 4th Edition, Page 332; Grants Method of Anatomy, 11th Edition, Page 244; Gray's Anatomy, 36th Edition, Page 563", 'reasoning': "Finding reasoning paths:\n1. Urogenital diaphragm -> Pelvic anatomy -> Muscles and fascia -> Deep perineal pouch\n2. Urogenital diaphragm -> Components of the perineum -> Muscles and connective tissue\n3. Urogenital diaphragm -> Anatomical structures -> Superficial vs. deep fascia\n\nReasoning Process:\n1. The urogenital diaphragm is a

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Load model and tokenizer
model_checkpoint = model_path
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 2. Example prompt construction
example = {
    "question": "What is the capital of France?",
    "choices": ["Berlin", "Madrid", "Paris", "Rome"],
    "answer": "Paris"  # Correct answer for later evaluation
}

# Build the prompt
prompt = f"Question: {example['question']}\n"
for idx, choice in enumerate(example['choices']):
    prompt += f"({chr(65 + idx)}) {choice}\n"
prompt += "\nAnswer:"

# 3. Tokenize
inputs = tokenizer(prompt, return_tensors="pt")

# 4. Generate output
outputs = model.generate(**inputs, max_new_tokens=10)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 5. See the generated text
print(generated_text)

# 6. You can then manually or programmatically check if 'Paris' appears in output


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 19.65it/s]


Question: What is the capital of France?
(A) Berlin
(B) Madrid
(C) Paris
(D) Rome

Answer: (C) Paris

Question: Which


In [11]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the entire MedReason dataset
dataset = load_dataset("UCSC-VLAA/MedReason")['train']

# Load tokenizer and model from local directory
model_path = "D:/HuggingFace/Models/granite-3.0-2b-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.eval()

# Evaluation loop
correct = 0
total = 0

for ex in tqdm(dataset, desc="Zero-shot Evaluation"):
    question = ex['question']
    options_text = ex['options']           # e.g., "Answer Choices:\nA. ...\nB. ...\nC. ...\nD. ..."
    correct_answer = ex['answer'].strip().lower()  # Full answer text

    # Create prompt for text generation
    prompt = f"Question: {question}\n{options_text}\nAnswer:"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    # Generate model output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode and extract the model's generated answer
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_answer = output_text[len(prompt):].strip().lower()

    # Check if model's output contains the correct full answer string
    if correct_answer in generated_answer:
        correct += 1
    total += 1

# Final accuracy
accuracy = correct / total
print(f"\nZero-shot Accuracy on MedReason: {accuracy:.2%}")



Zero-shot Evaluation:   0%|▏                                                  | 95/32682 [1:32:00<525:58:36, 58.11s/it]


KeyboardInterrupt: 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

NameError: name 'torch' is not defined

In [3]:
import torch
print(torch.version.cuda)  # should not be None
print(torch.cuda.is_available())  # should be True

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Ardahan\AppData\Roaming\Python\Python311\site-packages\torch\lib\cudnn_adv64_9.dll" or one of its dependencies.

In [7]:
!pip uninstall torch torchaudio torchvision -y


In [5]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-win_amd64.whl (2449.4 MB)


ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [4]:
import sys
print(sys.executable)

!pip show torch

C:\Python311\python.exe


In [8]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --target "D:\pytorch"

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached pillow-11.2.1-cp311-cp311-win_amd64.whl.metadata (9.1 kB)
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------------------------- 536.2/536.2 kB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 112.3 MB/s eta 0:00:26
    --------------------------------------- 0.0/2.9 GB 117.0 MB/s eta 0:00:25
    --------------------------------------- 0.1/2.9 GB 108.5 MB/s eta 0:00:27
   - -------------------------------------- 0.1/2.9 GB 109.8 MB/s eta 0:00:26
   - -------------------------------------- 0.1/2.9 GB 109.6 MB/s eta 0:00:26
   - -------------------------------------- 0.1/2.9 GB 106.6 MB/s eta 0:00:27
   -- ------------------------------------- 0.1/2.9 GB 107.7 MB/s eta 0:00:26
   -- ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
pandas 2.2.1 requires numpy<2,>=1.23.2; python_version == "3.11", but you have numpy 2.2.5 which is incompatible.


In [2]:
import sys
sys.path.append("D:/pytorch")

In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Ardahan\AppData\Roaming\Python\Python311\site-packages\torch\lib\cudnn_adv64_9.dll" or one of its dependencies.

In [5]:
!python --version
!pip --version
!python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu120
    

Python 3.11.5
pip 25.1 from C:\Users\Ardahan\AppData\Roaming\Python\Python311\site-packages\pip (python 3.11)

Looking in indexes: https://download.pytorch.org/whl/cu120


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu120 --extra-index-url https://pypi.org/simple --user

Looking in indexes: https://download.pytorch.org/whl/cu120, https://pypi.org/simple
  Using cached torch-2.7.0-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached torchvision-0.22.0-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.7.0-cp311-cp311-win_amd64.whl.metadata (6.7 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.7.0-cp311-cp311-win_amd64.whl (212.5 MB)
Using cached torchvision-0.22.0-cp311-cp311-win_amd64.whl (1.7 MB)
Using cached torchaudio-2.7.0-cp311-cp311-win_amd64.whl (2.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)

  Attempting uninstall: sympy

    Found existing installation: sympy 1.13.1

   ---------------------------------------- 0/4 [sympy]
   ---------------------------------------- 0/4 [sympy]
   ---------------------------------------- 0/4 [sympy]
    Uninstalling sympy-1.13.1:
   ---------------------------------------- 0/4 [sympy]
   ---------------------------------------- 0/4 [s

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [2]:
import torch
print(torch.__file__)                # Should be on D:
print(torch.cuda.is_available())    # True if CUDA is working


ModuleNotFoundError: No module named 'torch'

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

ModuleNotFoundError: No module named 'torch'